# YOLO11 Training - 4 Channel Multispectral Dataset

Notebook to train a model with YOLO11 on multispectral images (RGB + Thermal)

In [ ]:
from pathlib import Path
from ultralytics import YOLO
import torch
import yaml

DATASET_YAML = Path(r"../data/thermal_yolo/dataset.yaml")
PROJECT_NAME = "thermal_extra"
EXPERIMENT_NAME = "thermalbg_nano"
MODEL_NAME = "yolo11l-seg.pt"  # Use yolo11n-seg, yolo11s-seg, yolo11m-seg, etc.

In [ ]:
print("YOLO11 TRAINING SETUP - Multispectral Segmentation")

# Check CUDA
print(f"\nDevice Info:")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print(f"   Training on CPU")

# Load dataset configuration
with open(DATASET_YAML, 'r') as f:
    dataset_config = yaml.safe_load(f)

print(f"\nDataset:")
print(f"   Path: {dataset_config['path']}")
print(f"   Channels: {dataset_config.get('channels', 3)}")
print(f"   Classes: {dataset_config['nc']}")
print(f"   Names: {dataset_config['names']}")

# Check number of images
train_img_dir = Path(dataset_config['path']) / "train" / "images"
val_img_dir = Path(dataset_config['path']) / "val" / "images"
n_train = len(list(train_img_dir.glob("*.tiff")))
n_val = len(list(val_img_dir.glob("*.tiff")))
print(f"\nDataset Size:")
print(f"   Training: {n_train} images")
print(f"   Validation: {n_val} images")

# Load pre-trained YOLO11n-seg model
print("LOADING MODEL")
print(f"\nLoading model: {MODEL_NAME}")
model = YOLO(MODEL_NAME)
print(f"Model loaded!")
print(f"\nArchitecture: YOLO11 Segmentation")
print(f"Model parameters: {sum(p.numel() for p in model.model.parameters()):,}")

# Training configuration
print("TRAINING CONFIGURATION")
training_config = {
    'data': str(DATASET_YAML.absolute()),
    'epochs': 50,
    'imgsz': 1280,
    'batch': 4,  
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'project': PROJECT_NAME,
    'name': EXPERIMENT_NAME,
    # 'patience': 50,  # Early stopping
    'save': True,
    'save_period': 5,  # Save checkpoint every 5 epochs
    'cache': False, 
    'workers': 8,
    'optimizer': 'SGD',
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    'plots': True,
    'verbose': True,
    'fliplr': 0.5,
    # 'copy_paste': 0.3,
    # 'copy_paste_mode': 'flip',
    'scale': 0.3,
    'translate': 0.1,
    'degrees': 5.0,
    'hsv_h': 0.0,
    'hsv_s': 0.1,
    'hsv_v': 0.1,
    'copy_paste': 0.0,
    'mosaic': 0.0,
    'mixup': 0.0,
}
print(f"\nTraining parameters:")
for key, value in training_config.items():
    print(f"   {key}: {value}")

In [ ]:
print("STARTING TRAINING")
print(f"\nStarting training...")
print(f"   Results saved in: {PROJECT_NAME}/{EXPERIMENT_NAME}/")
print(f"   Logs and metrics in: {PROJECT_NAME}/{EXPERIMENT_NAME}/results.csv")

# Start training
results = model.train(**training_config)

print("TRAINING COMPLETED!")

# Path to best and last model
best_model_path = Path(PROJECT_NAME) / EXPERIMENT_NAME / "weights" / "best.pt"
last_model_path = Path(PROJECT_NAME) / EXPERIMENT_NAME / "weights" / "last.pt"

print(f"\nSaved models:")
print(f"   Best model: {best_model_path}")
print(f"   Last model: {last_model_path}")

print(f"\nTo visualize results:")
print(f"   - Plots: {PROJECT_NAME}/{EXPERIMENT_NAME}/")
print(f"   - Metrics: {PROJECT_NAME}/{EXPERIMENT_NAME}/results.csv")
print(f"   - Confusion matrix: {PROJECT_NAME}/{EXPERIMENT_NAME}/confusion_matrix.png")

print(f"\nTraining completed successfully!")